In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import date
import urllib

In [ ]:
data = [] # create an empty list to temporarily store data


#Define starting params
page = 1
query = 'berlusconi'
fromdate = '1984-01-02'

base_url = 'https://ricerca.repubblica.it/ricerca/repubblica?'


while True:
    
    params = {
    'query' : query,
    'fromdate' : fromdate,
    'todate' : today,
    'page' : page,
    'sortby' : 'adate'
    }

    url = base_url + urllib.parse.urlencode(params)
    
    response = requests.get(url)
    
    if response.status_code == 200:
        print(f"Searching {page} for {fromdate}")
        soup = BeautifulSoup(response.content, 'html.parser')
        articles = soup.find_all('article')
        for article in articles:
            title = article.find('a').text.strip()
            date = article.find('time')['datetime']
            url_article = article.find('a')['href']
            
            # Only save headlines with the word 'Berlusconi' or "Silvio".
            if 'berlusconi' in title.lower() or 'silvio' in title.lower():
                data.append({'Title': title, 'Date': date, 'URL': url_article})
                time.sleep(1)
        
        page += 1
    
    # Add condition for error 413 (max number of pages)
    elif response.status_code == 413:
        print("Error 414: no more pages available. Starting new search.")
        fromdate = date
        page = 1
    
    # In any other case, break out of the loop
    else:
        print("Error unknown")
        break




In [ ]:
df = pd.DataFrame(data)
df.drop_duplicates(keep=False, inplace=True)
df.to_csv('headlines.csv', index=False)